In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import pipeline

In [2]:
# Load the cleaned and preprocessed data
df = pd.read_csv('/Users/mtc/ADS/ADS 509/reddit-text-mining-project/data/reddit_preprocessed.csv')

# Preview processed text
df[['subreddit', 'processed_text']].head()

,subreddit,processed_text
0,science,new study african reserve find dehorn rhino cu...
1,science,lowcalorie diet increase risk depression overw...
2,science,people world likely favor dominant authoritari...
3,science,selfperceived physical attractiveness link str...
4,science,efficient mrna delivery rest t cell reverse hi...


In [ ]:
x = df['processed_text']
y = df['subreddit']

0          science
1          science
2          science
3          science
4          science
           ...    
1566    technology
1567    technology
1568    technology
1569    technology
1570    technology
Name: subreddit, Length: 1571, dtype: object

## Classification Model
- Predict subreddit (science vs. technology) based on processed Reddit post text
- Model options: Logistic Regression, Naive Bayes, SVM, etc.
- Steps:
  1. Train/test split
  2. TF-IDF vectorization
  3. Model training and evaluation

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state = 509, stratify = df['subreddit'])

tfidf = TfidfVectorizer(min_df = 10, ngram_range=(1,2), stop_words='english')
x_train_tf = tfidf.fit_transform(X_train)
x_test_tf = tfidf.transform(X_test)

In [39]:
training_pipeline = pipeline.Pipeline(
    steps=[('tfidf', TfidfVectorizer(stop_words="english")),
            ('model', LinearSVC(random_state=509, tol=1e-5))])

grid_param = [{
    'tfidf__min_df': [5, 10],
    'tfidf__ngram_range': [(1, 3), (1, 6)],
    'model__penalty': ['l2'],
    'model__loss': ['hinge'],
    'model__max_iter': [10000]
}, {
    'tfidf__min_df': [5, 10],
    'tfidf__ngram_range': [(1, 3), (1, 6)],
    'model__C': [1, 10],
    'model__tol': [1e-2, 1e-3]
}]

gridSearchProcessor = GridSearchCV(estimator=training_pipeline,
                                   param_grid=grid_param,
                                   cv=5)
gridSearchProcessor.fit(X_train, Y_train)

best_params = gridSearchProcessor.best_params_
best_params = gridSearchProcessor.best_params_
print("Best alpha parameter identified by grid search ", best_params)

best_result = gridSearchProcessor.best_score_
print("Best result identified by grid search ", best_result)

Best alpha parameter identified by grid search  {'model__loss': 'hinge', 'model__max_iter': 10000, 'model__penalty': 'l2', 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 3)}
Best result identified by grid search  0.8758078795927402


In [40]:
gridsearch_results = pd.DataFrame(gridSearchProcessor.cv_results_)
gridsearch_results[['rank_test_score', 'mean_test_score',
                    'params']].sort_values(by=['rank_test_score'])[:5]

,rank_test_score,mean_test_score,params
0,1,0.875808,"{'model__loss': 'hinge', 'model__max_iter': 10..."
1,1,0.875808,"{'model__loss': 'hinge', 'model__max_iter': 10..."
8,3,0.871011,"{'model__C': 1, 'model__tol': 0.001, 'tfidf__m..."
9,3,0.871011,"{'model__C': 1, 'model__tol': 0.001, 'tfidf__m..."
4,5,0.870214,"{'model__C': 1, 'model__tol': 0.01, 'tfidf__mi..."


In [43]:
y_pred = gridSearchProcessor.predict(X_test)

In [44]:
confusion_matrix(Y_test, y_pred)

array([[136,  15],
       [ 12, 152]])

In [45]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

     science       0.92      0.90      0.91       151
  technology       0.91      0.93      0.92       164

    accuracy                           0.91       315
   macro avg       0.91      0.91      0.91       315
weighted avg       0.91      0.91      0.91       315

